In [7]:
import os
import time
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import FinNews as fn
import re

# Dash modules
import dash
import dash_table
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go

from app import app
from apps.ind_css import *
from port_charts import *

# Set up jupyter proxy
# JupyterDash.infer_jupyter_proxy_config() 

%matplotlib inline

In [8]:
r2_df = pd.read_csv('assets/models/tyler_rf_daily_update/r2_df.csv',index_col=0)

In [9]:
r2_df

,Target 7,Feats 60,Target 120
r2,0.9831,0.9115,0.9448


In [19]:
app = JupyterDash(__name__,suppress_callback_exceptions=True,external_stylesheets=[dbc.themes.SUPERHERO])

server = app.server

model_dict = {'Random Forest Regressor 120/30': 'RF Reg_target_120_rebal_30_2017-01-01',
              'Random Forest Regressor 120/60': 'RF Reg_target_120_rebal_60_2017-01-01'
              # 'Random Forest Regressor 60/30': 'RF Reg_target_60_rebal_30_2017-01-01'
              # 'CNN Visual Pattern Recognition': '75percent_confidence_no_holding_15m_cnn'
             }
model_list = [key for key in model_dict.keys()]

r2_df = pd.read_csv('assets/models/tyler_rf_daily_update/r2_df.csv',index_col=0)
feat_df = pd.read_csv('assets/models/tyler_rf_daily_update/feature_importance.csv',index_col=0)
corr_df = pd.read_csv('assets/models/tyler_rf_daily_update/corr.csv',index_col=0)

app.layout = html.Div([
                dcc.Dropdown(id='model_filter',
                    options=[{'label': i, 'value': i} for i in model_list],
                    value='Random Forest Regressor 120/30'),
                dcc.Graph(id='model-metrics',style={'width':'45%','display':'inline-block'}),
                dcc.Graph(id='corr-chart',style={'width':'45%','display':'inline-block'}),
                dcc.Graph(id='feature-chart')
            ])



@app.callback(dash.dependencies.Output('model-metrics','figure'),
    [dash.dependencies.Input('model_filter','value')])
def model_metrics(model_filter):
    
    fig = go.Figure(go.Indicator(
                        mode = "number",
                        value = r2_df['Target 7'].values[0],
                        title = {"text": "Model R2 value:<br>",'font':{'size':20,'color':'white'}},
                        number = {'font':{'size':18,'color':'white'}},
                        domain = {'x': [0, 1], 'y': [0, 1]}
                        ))
    
    fig.update_layout(paper_bgcolor='rgba(0,0,0,0)')
    
    return fig

@app.callback(dash.dependencies.Output('feature-chart','figure'),
    [dash.dependencies.Input('model_filter','value')])
def model_metrics(model_filter):
    
    feats_df = feat_df[['Target 7','Features']].sort_values(by='Target 7',ascending=False)
    
    fig = go.Figure()
    
    fig = fig.add_trace(
            go.Bar(y=feats_df['Target 7'].values, x=feats_df['Features'].values)
            )
    fig.update_layout(title=f'Feature Importance for {model_filter}')
                      
#     fig.update_layout(paper_bgcolor='rgba(0,0,0,0)')

    return fig

@app.callback(dash.dependencies.Output('corr-chart','figure'),
    [dash.dependencies.Input('model_filter','value')])
def model_metrics(model_filter):

    fig = px.imshow(corr_df)
    
    fig.update_layout(title='Feature Correlation')
    
    return fig




if __name__ == '__main__':
    app.run_server(debug=True,port=8061)

Dash app running on http://127.0.0.1:8061/


### Notes
1. How do I set a relative height
2. Could add ticker - https://community.plotly.com/t/strip-ticker-label/48348/6


In [11]:
pd.read_csv('assets/models/jeff_multi_factor/mf_mlp.csv')

,loss,binary_accuracy,precision,recall,val_loss,val_binary_accuracy,val_precision,val_recall
0,0.782244,0.521322,0.528646,0.430998,0.721785,0.493617,0.480519,0.654867
1,0.777499,0.497868,0.500000,0.341826,0.715240,0.519149,0.500000,0.530973
2,0.751071,0.496802,0.498221,0.297240,0.709998,0.510638,0.488095,0.362832
3,0.720353,0.508529,0.519685,0.280255,0.708551,0.510638,0.484375,0.274336
4,0.718844,0.502132,0.508065,0.267516,0.706861,0.497872,0.454545,0.221239
...,...,...,...,...,...,...,...,...
145,0.610978,0.659915,0.792308,0.437367,0.690786,0.646809,0.647059,0.584071
146,0.610825,0.686567,0.800000,0.501062,0.694649,0.621277,0.615385,0.566372
147,0.610516,0.654584,0.759717,0.456476,0.679385,0.646809,0.647059,0.584071
148,0.623506,0.657782,0.760417,0.464968,0.682021,0.642553,0.643564,0.575221


In [ ]:
sector_plot(open_pos_df[open_pos_df['Date']=='2021-03-03'], '2021-03-03')